<a href="https://colab.research.google.com/github/Anjasfedo/eceg-lsb-lzw-huffman/blob/main/ECEG/eceg_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [63]:
import random

class Point:
    def __init__(self, x=None, y=None):
        self.x = x
        self.y = y

    def is_infinity(self):
        """Check if the point is the point at infinity."""
        return self.x is None and self.y is None

    def __eq__(self, other):
        """Custom equality check for Point objects."""
        if isinstance(other, Point):
            return self.x == other.x and self.y == other.y
        return False

    def __repr__(self):
        if self.is_infinity():
            return "Point at Infinity"
        return f"Point({self.x}, {self.y})"

class ECEG:
  def __init__(self):
    self.a = 6
    self.b = 7
    self.p = 61

  def elliptic_curve_equation(self, x):
    return (x**3 + self.a*x + self.b) % self.p

  def is_on_curve(self, x, y):
    return self.elliptic_curve_equation(x)  == (y**2) % self.p

  def generate_random_point(self):
    while True:
      x = random.randint(1, self.p - 1)
      y = random.randint(1, self.p - 1)
      if self.is_on_curve(x, y):
        return Point(x, y)

  # def calc_point_add(self, P, Q):
  #   R = Point()  # Initialize the result point R
  #   print(P, Q)

  #   slope = ((Q.y - P.y) / (Q.x - P.x))

  #   # Calculate Rx
  #   R.x = (slope**2 - P.x - Q.x)

  #   # Calculate Ry
  #   R.y = (slope * (P.x - R.x) - P.y)

  #   return R

  def calc_point_add(self, P, Q):
    """Calculate the addition of two points P and Q on the elliptic curve."""
    R = Point()  # Initialize the result point R

    if P.is_infinity():
        return Q
    if Q.is_infinity():
        return P

    # Handle the case where P and Q are inverses
    if P.x == Q.x and (P.y != Q.y or P.y == 0):
        return Point()  # Point at infinity

    # Calculate slope
    if P.x == Q.x and P.y == Q.y:
        # Point doubling
        slope = (3 * P.x**2 + self.a) * pow(2 * P.y, -1, self.p) % self.p
    else:
        # Regular point addition
        slope = (Q.y - P.y) * pow(Q.x - P.x, -1, self.p) % self.p

    # Calculate Rx
    R.x = (slope**2 - P.x - Q.x) % self.p

    # Calculate Ry
    R.y = (slope * (P.x - R.x) - P.y) % self.p

    return R

  def calc_point_doubling(self, P):
      """Calculate the point doubling 2P = P + P on the elliptic curve."""
      R = Point()  # Initialize the result point R

      if P.is_infinity() or P.y == 0:
          # Point at infinity for vertical tangent or zero y-coordinate
          return Point()

      # Calculate slope for point doubling
      slope = (3 * P.x**2 + self.a) * pow(2 * P.y, -1, self.p) % self.p

      # Calculate Rx
      R.x = (slope**2 - 2 * P.x) % self.p

      # Calculate Ry
      R.y = (slope * (P.x - R.x) - P.y) % self.p

      return R

  def calc_point_subtraction(self, P, Q):
      """Calculate the subtraction of two points P - Q on the elliptic curve."""
      # Negate Q to get -Q
      Q_neg = Point(Q.x, (-Q.y) % self.p)

      # Add P and -Q
      return self.calc_point_add(P, Q_neg)

  def calc_point_multiplication(self, P, k):
    """Calculate kP using the double-and-add method."""
    R = Point()  # Start with the point at infinity
    current_point = P

    while k > 0:
        if k % 2 == 1:
            # If the current bit is 1, add the current point
            R = self.calc_point_add(R, current_point)
        # Double the point
        current_point = self.calc_point_add(current_point, current_point)
        k //= 2  # Move to the next bit

    return R

In [78]:
import unittest

class TestECEG(unittest.TestCase):
    def setUp(self):
        # Initialize the elliptic curve object and generate a random point for testing
        self.elliptic = ECEG()
        self.random_point = self.elliptic.generate_random_point()

    def test_point_on_curve(self):
        """Test if a randomly generated point lies on the elliptic curve."""
        # Verify that the generated random point satisfies the curve equation
        self.assertTrue(self.elliptic.is_on_curve(self.random_point.x, self.random_point.y), "Point is not on the curve.")

    def test_addition_doubling(self):
        """Test if P + P equals 2P."""
        # Perform addition: P + P
        result_add = self.elliptic.calc_point_add(self.random_point, self.random_point)
        self.assertTrue(self.elliptic.is_on_curve(result_add.x, result_add.y), "Addition result is not on the curve.")

        # Perform doubling: 2P
        result_doubling = self.elliptic.calc_point_doubling(self.random_point)
        self.assertTrue(self.elliptic.is_on_curve(result_doubling.x, result_doubling.y), "Doubling result is not on the curve.")

        # Verify that P + P equals 2P
        self.assertEqual(result_add, result_doubling, "P + P does not equal 2P.")

    def test_calc_addition_subtraction(self):
        """Test if (P + P) - P equals P."""
        # Perform addition: P + P
        result_add = self.elliptic.calc_point_add(self.random_point, self.random_point)
        self.assertTrue(self.elliptic.is_on_curve(result_add.x, result_add.y), "Addition result is not on the curve.")

        # Perform subtraction: (P + P) - P
        result_subtraction = self.elliptic.calc_point_subtraction(result_add, self.random_point)
        self.assertTrue(self.elliptic.is_on_curve(result_subtraction.x, result_subtraction.y), "Subtraction result is not on the curve.")

        # Verify that (P + P) - P equals P
        self.assertEqual(result_subtraction, self.random_point, "(P + P) - P does not equal P")

    def test_calc_doubling_multiplication(self):
        """Test if 2P doubling 2P equals 4P."""
        # Perform first doubling: 2P
        result_doubling_first = self.elliptic.calc_point_doubling(self.random_point)
        self.assertTrue(self.elliptic.is_on_curve(result_doubling_first.x, result_doubling_first.y), "Doubling result first is not on the curve.")

        # Perform second doubling: 4P
        result_doubling_second = self.elliptic.calc_point_doubling(result_doubling_first)
        self.assertTrue(self.elliptic.is_on_curve(result_doubling_second.x, result_doubling_second.y), "Doubling result second is not on the curve.")

        # Calculate 4P using scalar multiplication
        k = 4
        result_multiplication = self.elliptic.calc_point_multiplication(self.random_point, k)

        self.assertTrue(self.elliptic.is_on_curve(result_multiplication.x, result_multiplication.y), "Multiplication result is not on the curve.")

        # Verify that 2P doubling 2P equals 4P
        self.assertEqual(result_doubling_second, result_multiplication, "2P doubling 2P does not equal 4P")

    def test_calc_addition_doubling_subtraction_multiplication(self):
        """Test if (2P + 2P + 2P) - (P + P) equals 4P."""
        # Generate a random point
        random_point = self.elliptic.generate_random_point()
        self.assertTrue(self.elliptic.is_on_curve(random_point.x, random_point.y), "Random point is not on the curve.")

        # Calculate (P + P)
        result_right = self.elliptic.calc_point_add(random_point, random_point)
        self.assertTrue(self.elliptic.is_on_curve(result_right.x, result_right.y), "Addition result is not on the curve.")

        # Calculate (2P)
        doubled_point = self.elliptic.calc_point_doubling(random_point)
        self.assertTrue(self.elliptic.is_on_curve(doubled_point.x, doubled_point.y), "Doubling result is not on the curve.")

        # Calculate (2P + 2P + 2P)
        result_left = self.elliptic.calc_point_add(doubled_point, doubled_point)
        result_left = self.elliptic.calc_point_add(result_left, doubled_point)
        self.assertTrue(self.elliptic.is_on_curve(result_left.x, result_left.y), "Result of 2P + 2P + 2P is not on the curve.")

        # Calculate (2P + 2P + 2P) - (P + P)
        result_subtraction = self.elliptic.calc_point_subtraction(result_left, result_right)
        self.assertTrue(self.elliptic.is_on_curve(result_subtraction.x, result_subtraction.y), "Subtraction result is not on the curve.")

        # Calculate 4P using multiplication
        k = 4
        result_multiplication = self.elliptic.calc_point_multiplication(random_point, k)
        self.assertTrue(self.elliptic.is_on_curve(result_multiplication.x, result_multiplication.y), "Multiplication result is not on the curve.")

        # Verify (2P + 2P + 2P) - (P + P) equals 4P
        self.assertEqual(result_subtraction, result_multiplication, "(2P + 2P + 2P) - (P + P) does not equal 4P")


# Run the unittest tests in Colab
if __name__ == "__main__":
    unittest.main(argv=[''], verbosity=2, exit=False)


test_addition_doubling (__main__.TestECEG)
Test if P + P equals 2P. ... ok
test_calc_addition_doubling_subtraction_multiplication (__main__.TestECEG)
Test if (2P + 2P + 2P) - (P + P) equals 4P. ... ok
test_calc_addition_subtraction (__main__.TestECEG)
Test if P - 2P equals P ... ok
test_calc_doubling_multiplication (__main__.TestECEG)
Test if 2P doubling 2P equals 4P. ... ok
test_point_on_curve (__main__.TestECEG) ... ok

----------------------------------------------------------------------
Ran 5 tests in 0.021s

OK


# Dump code

## test case

In [59]:
def test_calc_add_and_doubling():
    elliptic = ECEG()

    # Generate a valid random point
    random_point = elliptic.generate_random_point()
    print(f"Random Point: {random_point}")

    # Ensure the generated point lies on the curve
    assert elliptic.is_on_curve(random_point.x, random_point.y), "Generated point is not on the curve."

    # Perform point addition (P + P)
    result_add = elliptic.calc_point_add(random_point, random_point)
    print(f"Result of P + P (Addition): {result_add}")

    # Perform point doubling (2P)
    result_doubling = elliptic.calc_point_doubling(random_point)
    print(f"Result of 2P (Doubling): {result_doubling}")

    # Ensure both addition and doubling results lie on the curve
    assert elliptic.is_on_curve(result_add.x, result_add.y), "Addition result is not on the curve."
    assert elliptic.is_on_curve(result_doubling.x, result_doubling.y), "Doubling result is not on the curve."

    if (result_add == result_doubling):
      print("Test passed: Point addition and doubling produce the same result.")
    else:
      Exception("Test failed: Point addition and doubling do not produce the same result.")

# Run the test
test_calc_add_and_doubling()

Random Point: Point(28, 17)
Result of P + P (Addition): Point(41, 0)
Result of 2P (Doubling): Point(41, 0)


In [38]:
def test_calc_subtraction():
  elliptic = ECEG()
  random_point = elliptic.generate_random_point()
  print(random_point)

  result_add = elliptic.calc_point_add(random_point, random_point)
  print(result_add)

  result_subtraction = elliptic.calc_point_subtraction(result_add, random_point)
  print(result_subtraction)

  assert result_subtraction != random_point, "should be same"

test_calc_subtraction()

Point(42, 7)
Point(2, 37)
Point(42, 7)


In [39]:
def test_calc_multiplication():
  elliptic = ECEG()
  random_point = elliptic.generate_random_point()
  print(random_point)

  result_doubling = elliptic.calc_point_doubling(random_point)
  print(result_doubling)

  result_doubling_2 = elliptic.calc_point_doubling(result_doubling)
  print(result_doubling_2)

  result_multiplication = elliptic.calc_point_multiplication(random_point, 2)
  print(result_multiplication)

  assert result_multiplication != result_doubling_2, "should be same"

test_calc_multiplication()

Point(11, 1)
Point(26, 7)
Point(57, 38)
Point(26, 7)


In [42]:
5 * 2 * 2, 5 * 2

(20, 10)